In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Download the VADER lexicon 
nltk.download('vader_lexicon')

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
df_boa = pd.read_csv('../data/boa_cleaned.csv', parse_dates=['date'])
df_cbe = pd.read_csv('../data/cbe_cleaned.csv', parse_dates=['date'])
df_dashen = pd.read_csv('../data/dashen_cleaned.csv', parse_dates=['date'])
df = pd.concat([df_boa, df_cbe, df_dashen])
df.reset_index(inplace=True)
df.info()

In [ ]:
# Function to clean each review
def clean_text(text):
    if pd.isnull(text):
        return ""
    
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')  # removes all emojis
    
    # Remove non-ASCII characters (Amharic, etc.)
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # Remove non-alphanumeric characters (except basic punctuation)
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"-]', ' ', text)

    # Keep only letters and whitespace
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = text.lower()
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
def get_sentiment(text):
    """
    Compute sentiment for a given text.
    Returns:
      sentiment_label: 'positive', 'neutral', or 'negative'
      sentiment_score: compound score as a float
    """
    scores = sid.polarity_scores(text)
    compound = scores['compound']
    # Define thresholds for sentiment labels
    if compound >= 0.05:
        label = 'positive'
    elif compound <= -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return label, compound


In [ ]:
df['review_cleaned'] = df['review'].apply(clean_text)

In [ ]:
df['bank'].value_counts().plot(kind='bar')

In [ ]:
df[['sentiment_label', 'sentiment_score']] = df['review'].apply(lambda x: pd.Series(get_sentiment(x)))

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df,x='sentiment_label', hue='bank')
plt.xlabel('Sentiment Label')
plt.ylabel('Count')
plt.title('Sentiment labels for Dashen, CBE and Abyssinia banks')
plt.show()

In [ ]:
from transformers import pipeline

In [ ]:
"""
Preprocessing includes lemmatization and stopword removal, 
which improves keyword extraction quality.
"""
def preprocess_text(text):
    """
    Preprocess the input text:
    - Lowercase
    - Remove stopwords
    - Lemmatize
    - Keep only alphabetic tokens
    """
    # Check if the input is a string
    if not isinstance(text, str):
        return ""
    
    # Use spaCy to process the text
    doc = nlp(text.lower())
    
    # Token filtering: remove stopwords and non-alphabetic tokens, apply lemmatization
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    # Join tokens back to a string
    return " ".join(tokens)

In [ ]:
# Create a new column with cleaned text
df['cleaned_review'] = df['review'].apply(preprocess_text)

In [ ]:
def apply_text_cleaning(text):
    text = text.lower()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"-]', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
df['review_cleaned'] =  df['review'].apply(apply_text_cleaning)

In [ ]:
df[['review', 'cleaned_review','sentiment_label']].head(30)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=df,x='sentiment_label', hue='bank')
plt.xlabel('Sentiment Label')
plt.ylabel('Count')
plt.title('Sentiment labels for Dashen, CBE and Abyssinia banks')
plt.show()

In [ ]:
df.to_csv('../data/cleaned_data.csv')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Load your cleaned CSV which contains the critical 'review' column
df = pd.read_csv('../data/cleaned_data.csv')

# Step 1: Use TF-IDF Vectorizer to extract important keywords from each review
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
X = tfidf.fit_transform(df['review'])

# Step 2: Cluster the reviews into themes using KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Assign a theme label to each review from the clustering results
df['identified_theme'] = kmeans.labels_

# View top keywords for each cluster
terms = tfidf.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    top_terms = [terms[ind] for ind in order_centroids[i, :10]]
    print(f"Cluster {i} top keywords: {top_terms}")

# Step 3: Export the DataFrame with the theme labels to a new CSV file
df.to_csv('../data/cleaned_themed_reviews.csv', index=False)

In [ ]:
theme_counts = df.groupby(['bank', 'identified_theme']).size().reset_index(name='counts')

# Visualize with a bar plot
plt.figure(figsize=(12,6))
sns.barplot(data=theme_counts, x='identified_theme', y='counts', hue='bank')
plt.title("Theme Distribution per Bank")
plt.xlabel("Identified Theme")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()